In [ ]:
import random

class Room_Effect: #room effects that occur upon entering
    def __init__(self, name, description):
        self.name = name
        self.description = description
    
    def activate_effect(self, character):
        print(f"Upon entering the room, you find: {self.name} - {self.description}")
        if self.name == "the room on fire": #a way to force player to move more
            print(f"{character.name} suffers one light wound if they remain here")
        elif self.name == "the room under lockdown": #prevent player from going back to previous room
            print(f"{character.name} sees the door lock behind them")
        elif self.name == "a specimen staring back at you": #start encounter
            print(f"{character.name} is now in combat with") #add {enemy.name} later when they are made
        elif self.name == "a place to get your bearings": #neutral room
            print("there is nothing to worry about in this room.")

Room_Effect_Pool = [ #still need to code in effects
    Room_Effect("the room on fire", "if in room at end of turn, suffer one light wound"),
    Room_Effect("the room under lockdown", "unable to move to previous room"),
    Room_Effect("a specimen staring back at you", "starts an encounter"),
    Room_Effect("a place to get your bearings", "nothing happens")
]

class Item: #small and large items to be obtained (hold 2 large, any number of small)
    def __init__(self, name, size = 'small'):
        self.name = name
        self.size = size 

class Character_Inventory: #managable inventory system to add/remove gained items
    def __init__(self):
        self.small_items = [] 
        self.large_items = []
    
    def add_item(self, item):
        if item.size == 'large' and len(self.large_items) < 2:
            self.large_items.append(item)
            print(f"You pick up {item.name}. It looks too heavy for the bag, you'll have to put it in your hand.")
        elif item.size == 'small':
            self.small_items.append(item)
            print(f"You pick up {item.name} and add it to your bag.")
        else:
            print("You can can only carry two large items. You've only got two hands after all.")
    
    def remove_item(self, item):
        if item in self.large_items:
            self.large_items.remove(item)
            print(f"You drop the {item.name}.")
        elif item in self.small_items:
            self.small_items.remove(item)
            print(f"You drop the {item.name}.")
        else:
            print(f"{item.name} is not in your inventory.")
    
    def view_inventory(self): #need to add way for user to add/remove items. Can be done as free action
        print("You peer into your bag, finding: ")
        if self.small_items:
            for item in self.small_items:
                print(f" - {item.name} (Small)")
        if self.large_items:
            for item in self.large_items:
                print(f" - {item.name} (Large)")
        else:
            print("The bag you are holding appears to be empty. You should take some time to fill it.")

small_loot_pool = ["Bandages", "Bullet", "Duct Tape"] #need to figure out more?
large_loot_pool = ["Riot Shield", "Repair Kit", "Pistol"] 

class Room: #create rooms and keep track of whether they've been visited
    def __init__(self, room_id):
        self.room_id = room_id
        self.visited = False
        self.effect = None
        self.small_loot_count = 0
        self.large_loot_count = 0
        self.enemy = None

    def visit(self, character):
        if not self.visited:
            self.visited = True
            self.apply_random_effect(character)
            self.generate_loot()
    
    def apply_random_effect(self, character):
        self.effect = random.choice(Room_Effect_Pool)
        self.effect.activate_effect(character)


    def search_room(self, character):
        if self.small_loot_count > 0 or self.large_loot_count > 0:
            print(f"You search the room...")

            for _ in range(self.small_loot_count):
                loot_item = random.choice(small_loot_pool) 
                character.inventory.add_item(Item(loot_item, 'small'))

            for _ in range(self.large_loot_count):
                loot_item = random.choice(large_loot_pool) 
                character.inventory.add_item(Item(loot_item, 'large'))

            self.small_loot_count = 0
            self.large_loot_count = 0
        else:
            print("You do not find anything else in the room.")

    def generate_loot(self):
        self.small_loot_count = random.randint(1, 3) 
        self.large_loot_count = random.randint(0, 1) 

class Character_Class: #initialize character classes/weapons/challenges for extra items
    def __init__(self, name, weapon, challenges):
        self.name = name
        self.weapon = weapon
        self.inventory = Character_Inventory()
        self.challenges = challenges
        self.in_combat = False

    def complete_challenge(self, challenge_name): #saving the rest of this for later, might even take out if its too much
        if challenge_name in self.challenges and not self.challenges[challenge_name]['completed']:
            self.challenges[challenge_name]['completed'] = True
            reward_item = self.challenges[challenge_name]['reward']
            self.unlock_item(reward_item)
            print(f"Task '{challenge_name}' complete. The company grants you: {reward_item}.")
            print(f"You pocket the {reward_item}.")
        else:
            print(f"Task '{challenge_name}' already complete or does not exist.")

class Character_Creator:
    def create_character(class_name, name):
        class_map = {
            'Engineer' : ('Plasma Cutter', Character_Class, {
                'Upload Data' : {'completed': False, 'reward': 'Computer'}, #Scan bodies? Computer can be used in certain rooms to activate their ability
                'Repair Systems' : {'completed': False, 'reward': 'Toolkit'} #Repair like power or something. Toolkit for crafting? or easy repair of something?
            }),
            'Overseer' : ('Pistol', Character_Class, {
                'Clearance Check' : {'completed': False, 'reward': 'Access Pass'}, #Not sure how to implement this yet. Access Pass would open locked doors
                'Camera Check' : {'completed': False, 'reward': 'Data Pad'} #Check cameras. Data Pad could check cameras for info at any time
            }),
            'Scientist' : ('Experimental Rifle', Character_Class, {
                'Study Specimens' : {'completed': False, 'reward': 'Specimen Research Sheet'}, #scan a specimen in scan room. grants knowledge of a weakness
                'Craft with Flames' : {'completed': False, 'reward': 'Flamethrower'} #craft flamethrower, does random damage based on 2 d6 and makes enemy retreat
            }),
            'Security Guard' : ('Assault Rifle', Character_Class, {
                'Breach the Area' : {'completed': False, 'reward': 'Taser'}, #Explore 4 rooms. Taser allows escape from room without triggering encounter with 4? uses
                'Clear the Area' : {'completed': False, 'reward': 'Exoskeleton'} #Kill 2? enemies. Exoskeleton prevents x damage (need to design monsters still)
            })
        }

        if class_name in class_map:
            weapon, character_class, challenges = class_map[class_name]
            return character_class(name, weapon, challenges)
        else:
            print ("The company does not employ that profession.")
            return None

def explore_room(character): #room exploration and turn taking
    rooms = [Room(i) for i in range(1, 16)] #set of 15 rooms
    random.shuffle(rooms) #randomize rooms
    current_room_index = 0 #start from first room
    turn_counter = 0 #track actions

    while True: #action loop
        print(f"\nYou are currently in Room {rooms[current_room_index].room_id}.")
        rooms[current_room_index].visit(character)
        
        for _ in range(2):  #action choices, 2 per turn
            print("\nChoose your action:")
            print("1. Search the room")
            print("2. Fight an enemy")
            print("3. Move to the next room")
            print("4. Move Backward")
            print("5. Stay Put")
            print("6. Check Inventory (NO ACTION USED)")
            print("7. Quit Game")


            action_choice = input("What would you like to do? (1/2/3/4/5/6/7): ").strip()

            if action_choice == '1': #search room
                rooms[current_room_index].search_room(character)
                turn_counter += 1
                print(f"\nAction {turn_counter} completed: room searched")
            elif action_choice == '2': #combat
                print("You attempt to fight an enemy!")
                turn_counter += 1
                # ADD COMBAT
            elif action_choice == '3': #move forward
                if current_room_index < len(rooms) - 1:
                    current_room_index += 1
                    print(f"\nMoving to Room {rooms[current_room_index].room_id}...")
                    turn_counter += 1
                else:
                    print("You are at the last room. You can only go backward.")
            elif action_choice == '4': #move backward
                if current_room_index > 0:
                    current_room_index -= 1
                    print(f"\nMoving to Room {rooms[current_room_index].room_id}...")
                    turn_counter += 1
                else:
                    print("You are currently in the first room. You can only go forward.")
            elif action_choice == '5': #end turn
                print("You decide to stay put.")
                break
            elif action_choice == '6': #view inventory
                character.inventory.view_inventory()
            elif action_choice == '7': #break loop and end game
                print("You have abandoned the company.")
                return
            else:
                print("Invalid action. Please choose again.")

        print(f"\nEnd of turn {turn_counter}.")
        input("Press Enter to continue to the next turn...")
        
def class_pick():
    while True: #run loop until class is chosen or user chooses to break after selecting wrong
        print("Choose your character: ")
        print("1. Engineer")
        print("2. Overseer") 
        print("3. Scientist") 
        print("4. Security Guard")  

        user_choice = input("Please pick the number corresponding to your desired class: ")

        class_choices = {
            '1': 'Engineer',
            '2' : 'Overseer',
            '3' : 'Scientist',
            '4' : 'Security Guard' 
        }

        if user_choice in class_choices:
            class_name = class_choices[user_choice]
            name = input(f"What's your name?: ")
            character = Character_Creator.create_character(class_name, name)
            final_choice = input(f"So you are {character.name} the {class_name}?").lower()

            if final_choice == 'y' or final_choice == 'yes':
                print(f"Welcome {character.name} the {class_name}! \nPlease take the provided {character.weapon}.") 
                print("The company has been authorized to offer additional support should you seek it, but they require additional tasks to be completed in return.")
                print("The tasks are as follows: ")
                
                explore_room(character) #start exploration post character creation. need to add action system and other actions
 
                break
        else: 
            print("\nI'm sorry, you don't seem to be in our database.")
            try_again = input("Would you like to try again?: ").lower()

            if try_again != 'y' and try_again != 'yes':
                print ("Server Locking")
                break
class_pick()